In [2]:
import pandas as pd
from typing import Any, Optional
import pandas as pd
import re
import os
from datetime import datetime,timedelta

In [5]:
import chardet

In [6]:
full_path = os.getcwd()

In [18]:
with open(full_path.partition('notebooks')[0] + '/data/02_intermediate/customers_validated.csv', "rb") as f:
    result = chardet.detect(f.read(5000))
    print(result['encoding'])

ISO-8859-1


In [44]:
import warnings
warnings.filterwarnings("ignore")

## 1. Importing the Customers dataset

In [21]:

customers = pd.read_csv(full_path.partition('notebooks')[0] + '/data/02_intermediate/customers_validated.csv', encoding="ISO-8859-1")


In [23]:
customers.tail()


,index,NewId,CustomerSince,CustomerStatus,DateOfBirth,EmploymentStatus,Gender,MaritalStatus,Placebrth,CustType,...,Habliter,Province,District,LegalDocName1Id,LegalDocName1IdDescription,LegalIssDate,LegalExpDate,LegalIssAuth,AMLRiskRating,YrNetMonthlyIn
122207,122207,122208,2016-11-29,Private Client - Standard,1973-01-16,EMPLOYED,MALE,DIVORCED,ISRAEL,RETAIL,...,Curso Superior,CABO DELGADO,MAHATE,2.0,DIRE,20240726ý20250520ý20250520,20250725ý20280519ý20280519,MIGRACAOýMAIRRO MAHATEýREEF INVESTIMENTOS,Baixo,346500.0
122208,122208,122209,2021-07-23,Private Client - Standard,1978-01-10,EMPLOYED,MALE,SINGLE,ZIMBABWE,NaN,...,NaN,MAPUTO,MATOLA 'A',3.0,Passaporte,20170614,20270613,REPUBLIC OF ZIMBABWE,Medio,3000.0
122209,122209,122210,2017-12-04,Private Client - Standard,1964-01-16,MB,MALE,DIVORCED,HLATHIKHULU,RETAIL,...,NaN,NaN,NaN,3.0,Passaporte,20161013,20261012,NaN,Baixo,NaN
122210,122210,122211,2017-12-04,Corporate - Small,NaN,NaN,NaN,NaN,NaN,RETAIL,...,NaN,CIDADE DE MAPUTO,BAIRRO DE SOMMERSHIELD,24.0,Certidao da Conservatoria de Regist,20151120ý20171206,20991120ý20991206,CONSERVATORIAýDIRECCAO DE CULTURA,Baixo,NaN
122211,122211,122212,2012-06-21,Private Client - Standard,2007-07-24,NaN,MALE,SINGLE,MAPUTO,RETAIL,...,Ensino Primario,CIDADE DE MAPUTO,HULENE 'A',1.0,BI,20210728,20260727,DNIC MAPUTO,Elevado,NaN


In [26]:
customers.columns

Index(['index', 'NewId', 'CustomerSince', 'CustomerStatus', 'DateOfBirth',
       'EmploymentStatus', 'Gender', 'MaritalStatus', 'Placebrth', 'CustType',
       'Nationality', 'NoOfDependents', 'OcupationDesc', 'ResidenceCode',
       'ResidenceStatus', 'ResidenceType', 'SegmentId', 'SegGroup', 'Title',
       'TownCountry', 'BirthInCorpDate', 'CustType.1', 'IndustryId',
       'Habliter', 'Province', 'District', 'LegalDocName1Id',
       'LegalDocName1IdDescription', 'LegalIssDate', 'LegalExpDate',
       'LegalIssAuth', 'AMLRiskRating', 'YrNetMonthlyIn'],
      dtype='object')

We will select the columns that may be important for this problem and apply some feature engineering to them.

The columns used from the Customers dataset are: CustomerId, CustomerStatus, DateOfBirth, EmploymentStatus, MaritalStatus, CustType (Customer Type), Occupation, ResidencyType, SegGroup (Customer segment group), BirthInCorpDate (date the customer was registered in the system), AmlRiskRating (Anti-Money Laundering risk score), and YeNetMonthlyIn (estimated net monthly income).

In [36]:
customers=customers[['NewId','DateOfBirth', 'EmploymentStatus','MaritalStatus', 'OcupationDesc','SegGroup','BirthInCorpDate',"AMLRiskRating","YrNetMonthlyIn"]]

- for 

In [38]:
customers.tail()

,NewId,DateOfBirth,EmploymentStatus,MaritalStatus,OcupationDesc,SegGroup,BirthInCorpDate,AMLRiskRating,YrNetMonthlyIn
122207,122208,1973-01-16,EMPLOYED,DIVORCED,SOCIO/A-GERENTE,Personal,NaN,Baixo,346500.0
122208,122209,1978-01-10,EMPLOYED,SINGLE,PROFESSOR/A,Personal,NaN,Medio,3000.0
122209,122210,1964-01-16,MB,DIVORCED,EMPRESARIO/A,Personal,NaN,Baixo,NaN
122210,122211,NaN,NaN,NaN,NaN,Company,2015-11-20,Baixo,NaN
122211,122212,2007-07-24,NaN,SINGLE,MENOR DE IDADE,Personal,NaN,Elevado,NaN


In [39]:
ref_date=pd.to_datetime("2024-12-31")

In [40]:
ref_date

Timestamp('2024-12-31 00:00:00')

##### 1.1. Age

- Customers who belong to the segment group "Personal" have a DateOfBirth, while customers in the "Company" segment group have a BirthInCorpDate. To standardise this information, we will calculate the age of each customer at the time of the loan — understanding that for individuals, this means their biological age, and for companies, it refers to the number of years since incorporation.

In [45]:
customers["DateOfBirth"]=pd.to_datetime(customers["DateOfBirth"], errors="coerce")
customers["BirthInCorpDate"]=pd.to_datetime(customers['BirthInCorpDate'], errors="coerce")

In [ ]:
customers["DateOfBirthFilled"]=customers["DateOfBirth"].fillna(customers["BirthInCorpDate"])

In [73]:
customers["Age"]=((ref_date-customers["DateOfBirthFilled"]).dt.days // 365).fillna(0).astype(int)

##### 1.2. Is_Employed

In [54]:
customers["EmploymentStatus"].value_counts()

EmploymentStatus
EMPLOYED         28955
TPE               8809
MB                5329
LP                3169
SELF-EMPLOYED      935
UE                 716
OTHER              606
STUDENT            565
RET                319
UNEMPLOYED         241
RETIRED             25
Name: count, dtype: int64

In [55]:
employed_values = ["EMPLOYED", "SELF-EMPLOYED", "TPE", "MB", "LP"]
customers["Is_Employed"] = customers["EmploymentStatus"].isin(employed_values).astype(int)

##### 1.3. Is_Married

In [69]:
customers["Is_Married"] = customers["MaritalStatus"].isin(["MARRIED", "PARTNER"]).astype(int)

##### 1.4. Remove Nulls

In [67]:
customers=customers[~(customers["SegGroup"].isna())]

In [71]:
customers=customers.rename(columns={"NewId":"CustomerId"})

In [76]:
customers_features=customers[["CustomerId","SegGroup","AMLRiskRating","YrNetMonthlyIn","Age","Is_Employed","Is_Married"]]

In [77]:
customers_features


,CustomerId,SegGroup,AMLRiskRating,YrNetMonthlyIn,Age,Is_Employed,Is_Married
4,5,Company,Medio,NaN,12,1,0
5,6,Personal,Baixo,NaN,59,1,0
6,7,Company,Baixo,NaN,43,0,0
7,8,Personal,Elevado,50000.0,41,0,0
8,9,Personal,Elevado,0.0,50,0,1
...,...,...,...,...,...,...,...
122207,122208,Personal,Baixo,346500.0,51,1,0
122208,122209,Personal,Medio,3000.0,47,1,0
122209,122210,Personal,Baixo,NaN,61,1,0
122210,122211,Company,Baixo,NaN,9,0,0
